In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import datetime as dt

from sympy import symbols, Symbol, Eq, nsolve, solve, sqrt
from decimal import Decimal, getcontext

import multiprocessing as mp

from LP_Action_Liquidity_Parameter_Function import Get_Liquidity_Parameter, Get_Price

data_dir = '/Users/zhicong/Dropbox/DEXs'
pool_name = 'USDC-WETH 100 1'

# LP Action

We import all LP actions and conduct basic cleaning.

In [2]:
lp_action = pd.read_csv(os.path.join(data_dir, "uniswapv3_lp_actions", pool_name + ".csv"))
lp_action.columns = [x.upper() for x in lp_action.columns.tolist()]
lp_action.dropna(subset = ['AMOUNT0_USD','AMOUNT1_USD'], inplace = True)

# Final cleaning
lp_action['BLOCK_NUMBER'] = lp_action['BLOCK_NUMBER'].astype(int)
lp_action.sort_values(by = ['BLOCK_NUMBER'], ascending = False, inplace = True)
lp_action['AMOUNT0_ADJUSTED'] = lp_action['AMOUNT0_ADJUSTED'].astype(float)
lp_action['AMOUNT1_ADJUSTED'] = lp_action['AMOUNT1_ADJUSTED'].astype(float)
lp_action['BLOCK_TIMESTAMP'] = pd.to_datetime([x[:-5] for x in lp_action['BLOCK_TIMESTAMP'].astype(str)])
lp_action = lp_action[(lp_action['AMOUNT0_ADJUSTED'] != 0) | (lp_action['AMOUNT1_ADJUSTED'] != 0)]
lp_action['NF_TOKEN_ID'] = lp_action['NF_TOKEN_ID'].apply(str).str.replace('.0','', regex = False)

# Changing signs of addition and extraction
lp_action['ACTION'] = lp_action['ACTION'].replace({'INCREASE_LIQUIDITY':'Increase Liquidity',
                                                   'DECREASE_LIQUIDITY':'Decrease Liquidity'})
lp_action.rename(columns = {'LIQUIDITY_PROVIDER':'SENDER', 'ACTION':'EVENT_NAME'}, inplace = True)

# Get lower and upper price from tick
lp_action['PRICE_LOWER'] = lp_action['TICK_UPPER'].apply(Get_Price)
lp_action['PRICE_UPPER'] = lp_action['TICK_LOWER'].apply(Get_Price)

# Reverse to make sure LPs are sorted in the same order as they are executed
lp_action = lp_action.iloc[::-1]
lp_action.reset_index(drop = True, inplace = True)

print("Number of LP Actions in this pool:",len(lp_action))
lp_action.head()

Number of LP Actions in this pool: 1028


,POOL_NAME,TX_HASH,EVENT_NAME,BLOCK_NUMBER,BLOCK_TIMESTAMP,SENDER,NF_TOKEN_ID,AMOUNT0_ADJUSTED,TOKEN0_SYMBOL,AMOUNT0_USD,AMOUNT1_ADJUSTED,TOKEN1_SYMBOL,AMOUNT1_USD,TICK_LOWER,TICK_UPPER,PRICE_LOWER,PRICE_UPPER
0,USDC-WETH 100 1,0x9a773c2ab3acc47552e73d553b35e29c8fb0ca957688...,Increase Liquidity,13616454,2021-11-14 21:44:29,0x0ce37f7166d9f94180274348207829f72ce23737,153900,3.903388e+03,USDC,3.903388e+03,1.000000,WETH,4556.240000,191872,192146,4525.040389211555520887486636638641357421875,4650.7343552755000928300432860851287841796875
1,USDC-WETH 100 1,0x536c31d1ea4f5300214bcce3719bc418a8d0c045a884...,Increase Liquidity,13616513,2021-11-14 21:59:27,0x0ce37f7166d9f94180274348207829f72ce23737,153900,3.903388e+04,USDC,3.903388e+04,10.000000,WETH,45562.400000,191872,192146,4525.040389211555520887486636638641357421875,4650.7343552755000928300432860851287841796875
2,USDC-WETH 100 1,0x80c180d5b5fd5293241c574280632ef953ee1eb948d1...,Increase Liquidity,13616555,2021-11-14 22:07:56,0x0ce37f7166d9f94180274348207829f72ce23737,153900,5.000000e+05,USDC,5.000000e+05,135.317523,WETH,621785.369240,191872,192146,4525.040389211555520887486636638641357421875,4650.7343552755000928300432860851287841796875
3,USDC-WETH 100 1,0x199ffb9bf6d564b6e9267e1e8223152619d9b02fb40b...,Increase Liquidity,13616713,2021-11-14 22:45:01,0x0ce37f7166d9f94180274348207829f72ce23737,153918,6.717100e+05,USDC,6.717100e+05,0.000000,WETH,0.000000,192004,192087,4551.8156982596710804500617086887359619140625,4589.7510859065869226469658315181732177734375
4,USDC-WETH 100 1,0x0b48916af7a778673ddd57ba88ff7ad16256eb21f254...,Decrease Liquidity,13617092,2021-11-15 00:08:19,0x0ce37f7166d9f94180274348207829f72ce23737,153900,1.164073e+06,USDC,1.164073e+06,11.664559,WETH,53598.763959,191872,192146,4525.040389211555520887486636638641357421875,4650.7343552755000928300432860851287841796875


# Match Withdrawal with Deposition

In [3]:
"""
token_transfer = pd.read_csv(os.path.join(data_dir, "uniswapv3_NFT", pool_name + ".csv"))
token_transfer.columns = [x.upper() for x in token_transfer.columns.tolist()]
token_transfer = token_transfer.sort_values(by = ['TOKENID','BLOCK_NUMBER'])

token_transfer = token_transfer[token_transfer['BLOCK_NUMBER'] <= lp_action['BLOCK_NUMBER'].max()]

token_transfer.head()

initial = token_transfer.loc[token_transfer['NFT_FROM_ADDRESS'] == '0x0000000000000000000000000000000000000000',['TOKENID','TX_HASH','NFT_TO_ADDRESS']]
initial['EVENT_NAME'] = 'Increase Liquidity'

final = token_transfer.groupby(['TOKENID'])[['TOKENID','TX_HASH','NFT_TO_ADDRESS']].tail(1)
final['EVENT_NAME'] = 'Decrease Liquidity'

token_transfer = pd.concat([initial,final], axis = 0)
token_transfer.sort_values(by = ['TOKENID'], inplace = True)
token_transfer.rename(columns = {'NFT_TO_ADDRESS':'SENDER'}, inplace = True)

token_transfer.head()
"""

'\ntoken_transfer = pd.read_csv(os.path.join(data_dir, "uniswapv3_NFT", pool_name + ".csv"))\ntoken_transfer.columns = [x.upper() for x in token_transfer.columns.tolist()]\ntoken_transfer = token_transfer.sort_values(by = [\'TOKENID\',\'BLOCK_NUMBER\'])\n\ntoken_transfer = token_transfer[token_transfer[\'BLOCK_NUMBER\'] <= lp_action[\'BLOCK_NUMBER\'].max()]\n\ntoken_transfer.head()\n\ninitial = token_transfer.loc[token_transfer[\'NFT_FROM_ADDRESS\'] == \'0x0000000000000000000000000000000000000000\',[\'TOKENID\',\'TX_HASH\',\'NFT_TO_ADDRESS\']]\ninitial[\'EVENT_NAME\'] = \'Increase Liquidity\'\n\nfinal = token_transfer.groupby([\'TOKENID\'])[[\'TOKENID\',\'TX_HASH\',\'NFT_TO_ADDRESS\']].tail(1)\nfinal[\'EVENT_NAME\'] = \'Decrease Liquidity\'\n\ntoken_transfer = pd.concat([initial,final], axis = 0)\ntoken_transfer.sort_values(by = [\'TOKENID\'], inplace = True)\ntoken_transfer.rename(columns = {\'NFT_TO_ADDRESS\':\'SENDER\'}, inplace = True)\n\ntoken_transfer.head()\n'

Merge token_transfer into lp_action.

# Add Liquidity Parameter

In [4]:
with mp.Pool(10) as pool:
    result = pool.imap(Get_Liquidity_Parameter, lp_action[['AMOUNT1_ADJUSTED','AMOUNT0_ADJUSTED','PRICE_LOWER','PRICE_UPPER']].iterrows(), chunksize = 1)
    output = [x for x in result]

lp_action['L'] = output

Working on row: 0


In [5]:
lp_action.to_csv(os.path.join(data_dir, "uniswapv3_lp_action_LP", pool_name + "_LP.csv"), index=False)